In [ ]:
!pip install transformers[sentencepiece]
!pip install huggingface_hub
!pip install datasets

     |████████████████████████████████| 2.9 MB 7.8 MB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 1.2 MB 30.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 8.2 MB/s 
     |████████████████████████████████| 1.3 MB 33.4 MB/s 
     |████████████████████████████████| 243 kB 52.9 MB/s 
     |████████████████████████████████| 125 kB 56.5 MB/s 
     |████████████████████████████████| 160 kB 51.7 MB/s 
     |████████████████████████████████| 271 kB 56.4 MB/s 


In [ ]:
model_name = "deepset/roberta-base-squad2"
input_max_length = 384
doc_stride = 128
answer_max_lenght= 30

In [ ]:
import transformers
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/model1')

In [ ]:
from datasets import load_dataset
dataset_train = load_dataset('csv', data_files={'train':'/content/drive/MyDrive/train. csv'},split = 'train[:80%]')
dataset_validation = load_dataset('csv', data_files={'validation':'/content/drive/MyDrive/train. csv'},split='validation[-20%:]')

Using custom data configuration default-dd7f7578b263d377


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-dd7f7578b263d377/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


Using custom data configuration default-38e22be8055c6103


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-38e22be8055c6103/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


In [ ]:
dataset_train

In [ ]:
dataset_validation

In [ ]:
from transformers import  AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
def dataset_features_preparer(dataset):

    #Remove white spaces present in questions
    dataset["question"] = [q.lstrip() for q in dataset["question"]]

    #Tokenizing the context and question
    tokenized_dataset = tokenizer(
        dataset["question"],
        dataset["context"],
        truncation = "only_second",
        max_length = input_max_length,
        stride = doc_stride,
        return_overflowing_tokens = True,
        return_offsets_mapping = True,
        padding = "max_length",
    )

    #When lenght is greater than 384 it will generate another feature of same context so we need map each feature to its context
    sample_mapping = tokenized_dataset.pop("overflow_to_sample_mapping")

    #offset mapping --> map from first and last character position
    offset_mapping = tokenized_dataset.pop("offset_mapping")

    # Making output features
    tokenized_dataset["start_positions"] = []
    tokenized_dataset["end_positions"] = []


    for i, offsets in enumerate(offset_mapping):

        #Arrangment for impossible answers
        input_ids = tokenized_dataset["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_dataset.sequence_ids(i)

        #This is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers_start = dataset["answer_start"][sample_index]
        answers_end = dataset["answer_end"][sample_index]

        # If no answers are given, set the cls_index as answer.
        if  answers_start-answers_end == 0:
            tokenized_dataset["start_positions"].append(cls_index)
            tokenized_dataset["end_positions"].append(cls_index)
        else:
            
            start_char = answers_start
            end_char = answers_end

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1):
                token_end_index -= 1

            #If the answer is out of the span
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_dataset["start_positions"].append(cls_index)
                tokenized_dataset["end_positions"].append(cls_index)
            else:
                
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_dataset["start_positions"].append(token_start_index - 1)
                while token_end_index>=0 and offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_dataset["end_positions"].append(token_end_index + 1)

    return tokenized_dataset

In [ ]:
tokenized_datasets = dataset_train.map(dataset_features_preparer, batched=True, remove_columns=dataset_train.column_names)
tokenized_datasets_validation = dataset_validation.map(dataset_features_preparer, batched=True, remove_columns=dataset_validation.column_names)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets_validation

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
args = TrainingArguments(
    f"{model_name}-finetuned-qa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=15,
    per_device_eval_batch_size=15,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# trainer.train()

In [ ]:
#model.save_pretrained(save_directory='/content/drive/MyDrive/model1')

In [ ]:
import torch
import numpy as np
n_best_size = 20

In [ ]:
def prepare_validation_features(examples):
    
    examples["question"] = [q.lstrip() for q in examples["question"]]

    
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=input_max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    
    tokenized_examples["example_id"] = []
    tokenized_examples["offset_map"] = tokenized_examples["offset_mapping"]

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])


        tokenized_examples["offset_map"][i] = [
            (o if sequence_ids[k] == context_index else (-10, -10))
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    offset_mapping = tokenized_examples.pop("offset_mapping")

    return tokenized_examples

In [ ]:
dataset_sub = load_dataset('csv', data_files={'/content/drive/MyDrive/test. csv'})

Using custom data configuration default-7eb41a5d5d51c054
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-7eb41a5d5d51c054/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_sub = dataset_sub.rename_column(original_column_name='questions', new_column_name='question')

In [ ]:
dataset_sub

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question'],
        num_rows: 500
    })
})

In [ ]:
validation_features_mod2 = dataset_sub["train"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset_sub["train"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
validation_features_mod2.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'example_id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'offset_map': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

In [ ]:
validation_features_suicide = validation_features_mod2.remove_columns(column_names='offset_map')

In [ ]:
validation_features_mod2

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids', 'offset_map'],
    num_rows: 695
})

In [ ]:
validation_features_suicide

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids'],
    num_rows: 695
})

In [ ]:
raw_predictions = trainer.predict(validation_features_suicide)

The following columns in the test set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id.
***** Running Prediction *****
  Num examples = 695
  Batch size = 15


In [ ]:
raw_predictions

In [ ]:
validation_features_suicide.set_format(type=validation_features_suicide.format["type"], columns=list(validation_features_suicide.features.keys()))

In [ ]:
validation_features_suicide

In [ ]:
validation_features_mod2

In [ ]:
temp = validation_features_suicide.add_column(name= 'offset_mapping',column=validation_features_mod2["offset_map"])

In [ ]:
validation_features_mod2 = temp

In [ ]:
import collections
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):

    all_start_logits, all_end_logits = raw_predictions

    #map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)

    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    
    for example_index, example in enumerate(tqdm(examples)):

        feature_indices = features_per_example[example_index]

        min_null_score = None 
        valid_answers = []
        
        context = example["context"]

        for feature_index in feature_indices:

            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is (-10, -10)
                        or offset_mapping[end_index] is (-10, -10)
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(dataset_sub, validation_features_mod2, raw_predictions.predictions)

In [ ]:
final_predictions

In [ ]:
import csv

id, PredictionString = [], []

for key, value in final_predictions.items():
    id.append(key)
    PredictionString.append(value)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(id, PredictionString)),
               columns =['id', 'PredictionString'])
df

In [ ]:
count = 0
for i in range(500):
  if df["PredictionString"][i] == "":
    print(i)
    count = count+1
print("done")
print(count)

In [ ]:
df

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("squad")

In [ ]:
formatted_predictions = [{"prediction_text": v ,"id": k} for k, v in final_predictions.items()]
references = [{"answers": {'answer_start': [ex["answer_start"]], 'text' : ex["answer_text"]}, "id": ex["id"]} for ex in dataset_sub]

In [ ]:
metric.compute(predictions=formatted_predictions, references=references)